https://colab.research.google.com/github/SillyLossy/TavernAI-extras/blob/main/colab/GPU.ipynb<br>

Works with:<br>
KoboldAI https://github.com/KoboldAI/KoboldAI-Client<br>
Pygmalion https://huggingface.co/PygmalionAI/<br>
<br>
**Links**<br>
Cohee's TavernAI fork Github https://github.com/SillyLossy/TavernAI<br>
Cohee's TavernAI Extras Github  https://github.com/SillyLossy/TavernAI-extras/
TavernAI Discord https://discord.gg/zmK2gmr45t<br>
TavernAI Boosty https://boosty.to/tavernai<br>
Questions? Hit me up on Discord: Cohee#1207
<pre>
 Tavern.AI/ \ /  ^   ^ ^ ^    ~~~~ ^ \     /  ^ ^   ^ ^/ ^  ^ \/^  ^    \
         /^ ^\ ^  ^ ^   ^ ^  ~~   ^   \   /  ^  ^ ^   / ^ ^  ^/   ^ ^    \
        /^ ^ ^\^   ^ ^ ^   _||____   ^ \ /  ^  ^ ^   /       /  ^  ^  ^   \
 /\ /\ /\   ^  \  /\ /\   /\\\\\\\\   ^ \  ^ /\ /\ /\   /\ /\ /\  ^ ^  ^/\
//\\/\\/\\   ^  \//\\/\\ /__\\\\\\\\  _, \  //\\/\\/\\ //\\/\\/\\  ^ ^ //\\
//\\/\\/\\       //\\/\\ |__|_|_|__|   \__, //\\/\\/\\ //\\/\\/\\     ///\\\
 || || (@＾◡＾)(≖ ‸ ≖*) ( ←_← )\| /|   /\ \ヽ(°ㅂ°╬) |( Ψ▼ｰ▼)∈ (O_O; )  |||
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~  ~~~~~ ~~ 
</pre>
**Launch Instructions**<br>
1. Click the launch button.
2. Wait for the environment and model to load
3. After initialization, a TavernAI link will appear

**Faq**<br>
* Q: I do not get a TavernAI link
* A: It seems the localtunnel service is currently down, so the TavernAI link is unavailable. Need to wait for it to start working again.

In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
#Taken from KoboldAI colab
%%html
<b>Press play on the music player to keep the tab alive, then start TavernAI below (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title <b>TavernAI</b>
#@markdown <- Click For Start (≖ ‸ ≖ ✿)

Model = "Pygmalion 6B" #@param [ "Pygmalion 6B", "Pygmalion 6B Dev"] {allow-input: true}
Version = "Official" 
KoboldAI_Provider = "Cloudflare" #@param ["Localtunnel", "Cloudflare"]
use_google_drive = False #@param {type:"boolean"}
extras_enable_captioning = True #@param {type:"boolean"}
Captions_Model = "Salesforce/blip-image-captioning-large" #@param [ "Salesforce/blip-image-captioning-large", "Salesforce/blip-image-captioning-base", "microsoft/git-large-r-textcaps" ]
#@markdown * Salesforce/blip-image-captioning-large - good base model
#@markdown * Salesforce/blip-image-captioning-base - slightly faster but less accurate
#@markdown * microsoft/git-large-r-textcaps - works best for pictures with text (memes), but slightly worse for cartoon/anime drawings
extras_enable_emotions = True #@param {type:"boolean"}
Emotions_Model = "bhadresh-savani/distilbert-base-uncased-emotion" #@param ["bhadresh-savani/distilbert-base-uncased-emotion", "bhadresh-savani/bert-base-go-emotion"]
#@markdown * bhadresh-savani/distilbert-base-uncased-emotion = 6 supported emotions<br>
#@markdown * bhadresh-savani/bert-base-go-emotion = 28 supported emotions
extras_enable_memory = True #@param {type:"boolean"}
Memory_Model = "Qiliang/bart-large-cnn-samsum-ChatGPT_v3" #@param [ "Qiliang/bart-large-cnn-samsum-ChatGPT_v3", "Qiliang/bart-large-cnn-samsum-ElectrifAi_v10", "distilbart-xsum-12-3" ]
#@markdown * Qiliang/bart-large-cnn-samsum-ChatGPT_v3 - summarization model optimized for chats
#@markdown * Qiliang/bart-large-cnn-samsum-ElectrifAi_v10 - nice results so far, but still being evaluated
#@markdown * distilbart-xsum-12-3 - faster, but pretty basic alternative

Provider = KoboldAI_Provider
!nvidia-smi
import subprocess
import time
import sys
import os
import threading
import shutil
from google.colab import drive

    
if use_google_drive:
  drive.mount('/content/drive/')
  if not os.path.exists("/content/drive/MyDrive/TavernAI/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/characters/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/characters/")
  if not os.path.exists("/content/drive/MyDrive/TavernAI/chats/"):
    os.mkdir("/content/drive/MyDrive/TavernAI/chats/")
else:
  if not os.path.exists("/content/drive"):
    os.mkdir("/content/drive")
  if not os.path.exists("/content/drive/MyDrive/"):
    os.mkdir("/content/drive/MyDrive/")

def copy_characters(use_google_drive=False):
  if not use_google_drive:
    return
  
  src_folder = "/TavernAIColab/public/characters"
  dst_folder = "/content/drive/MyDrive/TavernAI/characters"

  for filename in os.listdir(src_folder):
    src_file = os.path.join(src_folder, filename)
    dst_file = os.path.join(dst_folder, filename)

    if os.path.exists(dst_file):
      print(f"{dst_file} already exists. Skipping...")
      continue

    shutil.copy(src_file, dst_folder)
    print(f"{src_file} copied to {dst_folder}")
Revision = ""

if Model == "Pygmalion 6B":
  Model = "PygmalionAI/pygmalion-6b"
  path = ""
  download = ""
  Version = "United"
elif Model == "Pygmalion 6B Dev":
  Model = "PygmalionAI/pygmalion-6b"
  Revision = "--revision dev"
  path = ""
  Version = "United"
  download = ""

if Provider == "Localtunnel":
  tunnel = "--localtunnel yes"
else:
  tunnel = ""



#Henk's KoboldAI script
!wget https://koboldai.org/ckds && chmod +x ckds
!./ckds --init only
if Provider == "Localtunnel":
  p = subprocess.Popen(['/content/ckds', '--model', Model, '--localtunnel', 'yes'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
else:
  p = subprocess.Popen(['/content/ckds', '--model', Model], stdout=subprocess.PIPE, stderr=subprocess.PIPE)


#Do not repeat! Tricks performed by a professional!
url = ''
while True:
    line = p.stdout.readline().decode().strip()
    if "KoboldAI has finished loading and is available at the following link: " in line:
        print(line)
        url = line.split("KoboldAI has finished loading and is available at the following link: ")[1]
        print(url)
        break
    if "KoboldAI has finished loading and is available at the following link for UI 1: " in line:
        print(line)
        url = line.split("KoboldAI has finished loading and is available at the following link for UI 1: ")[1]
        print(url)
        break
    if not line:
        break
    print(line)
    if "INIT" in line and "Transformers" in line:
      print("Model loading... (It will take 2 - 5 minutes)")

params = []

params.append('--cpu')

ExtrasModules = []

if (extras_enable_captioning):
  ExtrasModules.append('caption')

if (extras_enable_memory):
  ExtrasModules.append('summarize')

if (extras_enable_emotions):
  ExtrasModules.append('classify')

params.append(f'--classification-model={Emotions_Model}')
params.append(f'--summarization-model={Memory_Model}')
params.append(f'--captioning-model={Captions_Model}')
params.append(f'--enable-modules={",".join(ExtrasModules)}')

#TavernAI extras
%cd /
!git clone https://github.com/SillyLossy/TavernAI-extras
%cd TavernAI-extras
!pip install -r requirements.txt
!pip install tensorflow==2.11
cmd = f"python server.py {' '.join(params)}"
print(cmd)

extras_process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd='/TavernAI-extras', shell=True)
print('processId:', extras_process.pid)

while True:
    line = extras_process.stdout.readline().decode().strip()
    if "Running on " in line:
        break
    if not line:
        print('breaking on line')
        break
    print(line)

subprocess.call('nohup lt --port 5100 > ./extras.out 2> ./extras.err &', shell=True)
print('Waiting for lt init...')
time.sleep(5)

while True:
  if (os.path.getsize('./extras.out') > 0):
    with open('./extras.out', 'r') as f:
      lines = f.readlines()
      for x in range(len(lines)):
        if ('your url is: ' in lines[x]):
          print('TavernAI Extensions URL:')
          extras_url = lines[x].split('your url is: ')[1]
          print(extras_url)
      break
  if (os.path.getsize('./extras.err') > 0):
    with open('./extras.err', 'r') as f:
      print(f.readlines())
      break

#TavernAI
%cd /
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.37.2/install.sh | bash
!nvm install 19.1.0
!nvm use 19.1.0
!node -v
!git clone -b dev https://github.com/SillyLossy/TavernAI
copy_characters(use_google_drive)
%cd TavernAIColab
!npm install
time.sleep(1)
%env colab=2
%env colaburl=$url
if use_google_drive:
  %env googledrive=2
!nohup node server.js &
time.sleep(3)
print('KoboldAI LINK:')
print(url)
print('')
print('### TavernAI Extensions LINK ###')
print(extras_url)
print('')
print('###TavernAI LINK###')
!lt --port 8000
